# Tanscripting Video Files From TikTok

#### Installing dependencies
We need moviepy to convert the videos to mp3
We need openai-whisper to tanscript the files to text

In [ ]:
%pip install moviepy
%pip install -U openai-whisper
%pip install faster-whisper 
%pip install pandas 
%pip install requests

Importing

In [15]:
from moviepy import VideoFileClip
import whisper
import csv
import requests
from faster_whisper import WhisperModel
import httpx
import json

Downloading video files

In [ ]:
with open("tiktok_vids.json", "r") as file: 
    data = json.load(file)

links = []
for idx,post in enumerate(data):
    if "videoMeta" in post and "downloadAddr" in post["videoMeta"]:
        url = post["videoMeta"]["downloadAddr"]
        if url: 
            video_response = httpx.get(url)
            file_path = f"/Users/noursafadi/Documents/Uni/Parsons-Spring-25/Major Studio 02/Thesis/tiktok-scraper/videos/{idx}.mp4"
            with open(file_path, "wb") as f:
                f.write(video_response.content)
            print(f"{idx}: Seccessful")
        else:
            print(f"{idx} is not Available")
    else:
        print(f"{idx} is Not Available")

Converting from video to audio

In [ ]:
video_path = "/content/drive/MyDrive/Thesis/Code/download.mp4"
audio_output_path = "/content/drive/MyDrive/Thesis/Code/my_audio.mp3"

video = VideoFileClip(video_path)
audio = video.audio
audio.write_audiofile(audio_output_path)

video.close()

In [ ]:
#OpenAI WHISPER basic transcripting
model = whisper.load_model("turbo")
result = model.transcribe(my_audio, task="translate")
print(result["text"])

In [ ]:

# 🔑 Set Your DeepL API Key Here
DEEPL_API_KEY = "39f5e8f7-4926-44de-b567-420c316bc88e:fx"  # Replace with your actual key

# Define DeepL Translation Function
def translate_text(text, source_lang="DE", target_lang="EN"):
    url = "https://api-free.deepl.com/v2/translate"  # Use "api.deepl.com" for Pro accounts
    params = {
        "auth_key": DEEPL_API_KEY,
        "text": text,
        "source_lang": source_lang,
        "target_lang": target_lang
    }
    response = requests.post(url, data=params)
    if response.status_code == 200:
        return response.json()["translations"][0]["text"]
    else:
        print("Translation Error:", response.text)
        return text  # Return original text in case of failure

# Load Whisper Model
model = WhisperModel("medium")  # Use "small", "large-v3", etc., depending on need

# Transcribe the Audio
segments, _ = model.transcribe(my_audio, word_timestamps=True)

# Define Output CSV File
csv_filename = "/content/drive/MyDrive/Thesis/Code/Translated.csv"

# Open CSV File for Writing
with open(csv_filename, mode="w", newline="", encoding="utf-8") as file:
    writer = csv.writer(file)

    # Write Header
    writer.writerow(["Segment Start", "Segment End", "German Text", "English Translation"])

    # Process Each Transcribed Segment
    for segment in segments:
        english_translation = translate_text(segment.text)
        writer.writerow([segment.start, segment.end, segment.text, english_translation])

print(f"Transcription and translation saved to {csv_filename}")
